# Colab config

In [0]:
from google.colab import drive
drive.mount('drive', force_remount=True)

In [0]:
!pip install hyperas

## Imports

In [0]:
import os

import time

import gc

import numpy as np

from scipy import interp

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, roc_curve, auc

from keras import backend as K
from keras import optimizers
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Dropout, Input
from keras.losses import binary_crossentropy
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model, model_to_dot

## GPU configuration

In [0]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
set_session(tf.Session(config=config))
session = tf.Session(config=config)

# Hyperparameters optimization

Func **data()** required by Hyperas. It must return 4 arrays X_train, Y_train, X_test, Y_test.

The data for optimization are on four numpy files, generated with the notebook **import_images.ipynb**.

In [0]:
'''5-first-repeat'''

def data():
  prefix = "/content/drive/My Drive/Pesquisa - Dicom images/data"
  X_train = np.load(prefix + "/nps/solid-nodules/data-5-first-repeat/X_train.npy")
  X_test = np.load(prefix + "/nps/solid-nodules/data-5-first-repeat/X_test.npy")
  Y_train = np.load(prefix + "/nps/solid-nodules/data-5-first-repeat/Y_train.npy")
  Y_test = np.load(prefix + "/nps/solid-nodules/data-5-first-repeat/Y_test.npy")
  
  return X_train, Y_train, X_test, Y_test

## Space of search definition

* **conv_layer1**: [32, 48, 64, 96] unities
* **dense_layer1**: [32, 64, 128, 256] unities
* **dense_layer2**: [8, 16, 24, 32] unities
* **dropout** on dense layers: between 0 and 0.5

In [0]:
conv1 = [32, 48, 64, 96]
dense1 = [64, 96, 128]
dense2 = [16, 24, 32]
    
def model(X_train, Y_train, X_test, Y_test):
    conv1 = {{choice([32, 48, 64, 96])}}
    dense1 = {{choice([32, 64, 128])}}
    dense2 = {{choice([16, 24, 32])}}
    
    input_layer = Input(X_train.shape[1:5])
    
    conv_layer1 = Conv3D(conv1, kernel_size=(3, 3, 3), activation='relu')(input_layer)
    pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer1)
    
    flatten_layer = Flatten()(pooling_layer1)
    
    dense_layer1 = Dense(dense1, activation='relu')(flatten_layer)
    dense_layer1 = Dropout({{uniform(0, .5)}})(dense_layer1)
    
    dense_layer2 = Dense(dense2, activation='relu')(dense_layer1)
    dense_layer2 = Dropout({{uniform(0, .5)}})(dense_layer2)
    
    output_layer = Dense(units=1, activation='sigmoid')(dense_layer2)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    opt = optimizers.RMSprop(lr=0.0001)
    
    model.compile(loss=binary_crossentropy, optimizer=opt, metrics=['acc'])

    model.fit(X_train, Y_train,
              batch_size=128,
              epochs=10,
              verbose=2,
              validation_data=(X_test, Y_test)
              )
    
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    
    print('Test accuracy:', acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

## Otimization

In [8]:
X_train, Y_train, X_test, Y_test = data()

start = time.time()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      verbose=False,
                                      notebook_name='drive/My Drive/Pesquisa - Dicom images/notebooks/optimizer_5_first_repeat')
end = time.time()

print("Time on optimization:", (end - start)/60, "minutos")

print()
print('Summary of the best model: -----------------------')

print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))

print(best_model.summary())

c1 = conv1[best_run['conv1']]
d1 = dense1[best_run['dense1']]
d2 = dense2[best_run['dense2']]
drop1 = best_run['Dropout']
drop2 = best_run['Dropout_1']

print('Conv 1:', c1, 'unidades')
print('Dense 1:', d1, 'unidades')
print('Dense 2:', d2, 'unidades')
print('Dropout 1:', drop1)
print('Dropout 2:', drop2)

  0%|          | 0/30 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 5646 samples, validate on 100 samples
Epoch 1/10
 - 6s - loss: 0.8592 - acc: 0.6971 - val_loss: 0.6018 - val_acc: 0.7200

Epoch 2/10
 - 3s - loss: 0.5872 - acc: 0.7304 - val_loss: 0.4254 - val_acc: 0.7600

Epoch 3/10
 - 3s - loss: 0.5518 - acc: 0.7320 - val_loss: 0.4448 - val_acc: 0.7800

Epoch 4/10
 - 3s - loss: 0.5464 - acc: 0.7340 - val_loss: 0.4956 - val_acc: 0.7400

Epoch 5/10
 - 3s - loss: 0.5243 - acc: 0.7457 - val_loss: 0.4956 - val_acc: 0.7700

Epoch 6/10
 - 3s - loss: